In [1]:
!pip install mysql-connector-python
!pip install sqlalchemy
!pip install pymysql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.4/27.4 MB 74.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00


In [6]:
import pandas as pd
import sqlalchemy
import pymysql
from datetime import datetime,timedelta

In [12]:
yesterday_date = datetime.now() - timedelta(days=1)
prev_date = yesterday_date.strftime("%Y:%m:%d")+" 00:00:00"
#print(prev_date)

In [3]:
username = "pal-dev"
password = "admin123"
endpoint = "18.133.242.66"
source_index = 'mw-index-tasker'
destination_index = 'mw-index-pal'


db_name = "miweigh_db"
#table_name = "clients"

# create the connection string to connect to the RDS instance
conn = pymysql.connect(host=endpoint, user=username,
                       passwd=password, db=db_name)

In [13]:
query = f"select distinct Op_Name from operators_performances where indexName = '{source_index}' AND date = '{prev_date}';"
op_names = pd.read_sql_query(query, conn)
popular_names = [
    "Oliver", "Jack", "Harry", "George", "Noah", "Leo", "Arthur", "Muhammad", "Oscar", "Charlie",
    "Olivia", "Amelia", "Isla", "Ava", "Emily", "Sophia", "Grace", "Mia", "Lily", "Freya",
    "William", "Noah", "Liam", "James", "Oliver", "Benjamin", "Lucas", "Henry", "Alexander", "Mason",
    "Emma", "Sophia", "Olivia", "Ava", "Isabella", "Mia", "Amelia", "Charlotte", "Harper", "Lily",
    "Liam", "Noah", "Jackson", "Lucas", "Logan", "Jacob", "Oliver", "Ethan", "Aiden", "Caden",
    "Sophia", "Emma", "Olivia", "Ava", "Isabella", "Mia", "Charlotte", "Amelia", "Harper", "Evelyn",
    "George", "Arthur", "Harry", "Jack", "Freddie", "Oscar", "Thomas", "Charlie", "Leo", "Henry",
    "Isabella", "Sophia", "Mia", "Charlotte", "Amelia", "Harper", "Evelyn", "Abigail", "Emily", "Lily",
    "Mason", "Liam", "Noah", "Lucas", "Logan", "Oliver", "Ethan", "Caden", "Benjamin", "James",
    "Emily", "Lily", "Amelia", "Sophia", "Olivia", "Ava", "Mia", "Isabella", "Charlotte", "Grace",
    "MuhammadUsman", "Oliver", "Noah", "George", "Leo", "Jack", "Arthur", "Harry", "Muhammed", "Charlie"
]


<ipython-input-13-605cbc00e213>:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  op_names = pd.read_sql_query(query, conn)


In [14]:
op_names

,Op_Name
0,ADILE ARASLANOVA
1,ALEXANDRA VISAN
2,ATIKE CHAVDAROVA
3,CARMEN PATIERA
4,IONELA BIMBIRICA
5,MICHAELA MICHI
6,SC 3
7,SC 7
8,TSVETELINA


In [15]:
engine = sqlalchemy.create_engine(f'mysql+pymysql://{username}:{password}@{endpoint}/miweigh_db')

# Insert the DataFrame into the SQL table
table_name = 'operators_performances'  # Replace with your table name


In [16]:
name_dict = dict(zip(op_names.Op_Name, popular_names))

In [17]:
for prev_name in op_names.Op_Name:
  # if prev_name == 'A Rafalska':
  #   continue
  # print(prev_name)
  new_query = f"SELECT * FROM operators_performances WHERE indexName = '{source_index}' AND Op_Name = '{prev_name}';"
  values = pd.read_sql_query(new_query, conn)
  values.Op_Name = name_dict[prev_name]
  values.indexName = destination_index
  values['id'] =  values['id'].str.replace(prev_name, name_dict[prev_name])
  values.to_sql(table_name, con=engine, if_exists='append', index=False)
  
engine.dispose()

<ipython-input-17-8fbcea7623f6>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  values = pd.read_sql_query(new_query, conn)
<ipython-input-17-8fbcea7623f6>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  values = pd.read_sql_query(new_query, conn)
<ipython-input-17-8fbcea7623f6>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  values = pd.read_sql_query(new_query, conn)
<ipython-input-17-8fbcea7623f6>:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 c